In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# DATA

## ERROR 

### RUTA DE ACCESO QUERY 1

In [2]:
query_error_previo = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\CUBETAS\2024\01. ENERO\QUERIES\ERROR ENERO 2024.csv',dtype={'REPORT_ID':object,'AMOUNT':float})
query_error_previo.columns = query_error_previo.columns.str.strip().str.lower() 
query_error_previo.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
# query_sin_clabe= query_sin_clabe.columns.str.lower()
# query_sin_clabe.columns= query_sin_clabe.columns.str.strip()
query_error_previo['report_date_local']= pd.to_datetime(query_error_previo['report_date_local']).dt.tz_localize(None)
query_error_previo['ultimo_mov']= pd.to_datetime(query_error_previo['ultimo_mov'])
query_error_previo['merchant_id_query'] = query_error_previo['merchant_id']
query_error_previo['error_description'].fillna('sin_descripcion',inplace=True)
query_error_previo['index'] = (query_error_previo['merchant_id'].astype(str).replace(' ', '',regex=True)+'-'+
                               query_error_previo['amount'].astype(str).replace(' ', '',regex=True))
query_error_previo.loc[query_error_previo['error_description'].str.contains('Bank account information not available'), 'esquema_query'] = 'SIN_CLABE'
query_error_previo.loc[~query_error_previo['error_description'].str.contains('Bank account information not available'), 'esquema_query'] = 'ERROR_USUAL'
query_error_previo['ledger_flag'] =  query_error_previo['report_id'].astype(str).str[0]
query_error_previo.head()

,type,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,merchant_id_query,index,esquema_query,ledger_flag
0,DAILY,858342a7-fd31-40a4-8cc1-8e360cb12698,FARMACIA FARMA FA,M289RR3NC,2022-11-06,1.80,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-11-07,858342a7-fd31-40a4-8cc1-8e360cb12698,858342a7-fd31-40a4-8cc1-8e360cb12698-1.8,ERROR_USUAL,M
1,DAILY,da3a4b3a-7b06-49f1-8f59-ae60feececdc,El rincon del cantante,TNDPS2WWB,2022-12-14,0.95,ERROR,Bank account information not available,2022-12-14,da3a4b3a-7b06-49f1-8f59-ae60feececdc,da3a4b3a-7b06-49f1-8f59-ae60feececdc-0.95,SIN_CLABE,T
2,INSTANT,43685a7e-2583-480a-94be-f6aa7cfee8f6,NAILKERY SAN ANGELO,FYMY32LCT,2021-12-26,373.71,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2021-12-28,43685a7e-2583-480a-94be-f6aa7cfee8f6,43685a7e-2583-480a-94be-f6aa7cfee8f6-373.71,ERROR_USUAL,F
3,DAILY,3681aa22-8083-43a4-a552-ad7372081221,Bloombelle Lashes,RPB8JVFZD,2018-09-03,47.91,ERROR,failed swap,2018-09-03,3681aa22-8083-43a4-a552-ad7372081221,3681aa22-8083-43a4-a552-ad7372081221-47.91,ERROR_USUAL,R
4,DAILY,2fdcf827-4e1f-4b32-aa66-bdff89af941e,Bros Chicken,B484H8HYR,2021-02-21,362.21,ERROR,DEV.SPEICUENTA NO PERTE A BCO,2021-02-23,2fdcf827-4e1f-4b32-aa66-bdff89af941e,2fdcf827-4e1f-4b32-aa66-bdff89af941e-362.21,ERROR_USUAL,B


In [3]:
query_error_previo.shape

(157709, 13)

In [4]:
query_error_previo['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [5]:
query_error_previo['status'].unique()

array(['ERROR', 'ERROR_TV'], dtype=object)

### ERROR SETTLEMENT

In [6]:
query_error_previo_sett = query_error_previo[(query_error_previo['status'] == 'ERROR') |
                                             (query_error_previo['status'] == 'ERROR_TV')]
query_error_previo_sett.head()

,type,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,merchant_id_query,index,esquema_query,ledger_flag
0,DAILY,858342a7-fd31-40a4-8cc1-8e360cb12698,FARMACIA FARMA FA,M289RR3NC,2022-11-06,1.80,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-11-07,858342a7-fd31-40a4-8cc1-8e360cb12698,858342a7-fd31-40a4-8cc1-8e360cb12698-1.8,ERROR_USUAL,M
1,DAILY,da3a4b3a-7b06-49f1-8f59-ae60feececdc,El rincon del cantante,TNDPS2WWB,2022-12-14,0.95,ERROR,Bank account information not available,2022-12-14,da3a4b3a-7b06-49f1-8f59-ae60feececdc,da3a4b3a-7b06-49f1-8f59-ae60feececdc-0.95,SIN_CLABE,T
2,INSTANT,43685a7e-2583-480a-94be-f6aa7cfee8f6,NAILKERY SAN ANGELO,FYMY32LCT,2021-12-26,373.71,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2021-12-28,43685a7e-2583-480a-94be-f6aa7cfee8f6,43685a7e-2583-480a-94be-f6aa7cfee8f6-373.71,ERROR_USUAL,F
3,DAILY,3681aa22-8083-43a4-a552-ad7372081221,Bloombelle Lashes,RPB8JVFZD,2018-09-03,47.91,ERROR,failed swap,2018-09-03,3681aa22-8083-43a4-a552-ad7372081221,3681aa22-8083-43a4-a552-ad7372081221-47.91,ERROR_USUAL,R
4,DAILY,2fdcf827-4e1f-4b32-aa66-bdff89af941e,Bros Chicken,B484H8HYR,2021-02-21,362.21,ERROR,DEV.SPEICUENTA NO PERTE A BCO,2021-02-23,2fdcf827-4e1f-4b32-aa66-bdff89af941e,2fdcf827-4e1f-4b32-aa66-bdff89af941e-362.21,ERROR_USUAL,B


In [7]:
query_error_previo_sett.shape

(157709, 13)

In [8]:
query_error_previo_sett['status'].unique()

array(['ERROR', 'ERROR_TV'], dtype=object)

### ERROR COMPLETO

In [9]:
query_error = pd.concat([query_error_previo_sett])
query_error.head()

,type,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,merchant_id_query,index,esquema_query,ledger_flag
0,DAILY,858342a7-fd31-40a4-8cc1-8e360cb12698,FARMACIA FARMA FA,M289RR3NC,2022-11-06,1.80,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-11-07,858342a7-fd31-40a4-8cc1-8e360cb12698,858342a7-fd31-40a4-8cc1-8e360cb12698-1.8,ERROR_USUAL,M
1,DAILY,da3a4b3a-7b06-49f1-8f59-ae60feececdc,El rincon del cantante,TNDPS2WWB,2022-12-14,0.95,ERROR,Bank account information not available,2022-12-14,da3a4b3a-7b06-49f1-8f59-ae60feececdc,da3a4b3a-7b06-49f1-8f59-ae60feececdc-0.95,SIN_CLABE,T
2,INSTANT,43685a7e-2583-480a-94be-f6aa7cfee8f6,NAILKERY SAN ANGELO,FYMY32LCT,2021-12-26,373.71,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2021-12-28,43685a7e-2583-480a-94be-f6aa7cfee8f6,43685a7e-2583-480a-94be-f6aa7cfee8f6-373.71,ERROR_USUAL,F
3,DAILY,3681aa22-8083-43a4-a552-ad7372081221,Bloombelle Lashes,RPB8JVFZD,2018-09-03,47.91,ERROR,failed swap,2018-09-03,3681aa22-8083-43a4-a552-ad7372081221,3681aa22-8083-43a4-a552-ad7372081221-47.91,ERROR_USUAL,R
4,DAILY,2fdcf827-4e1f-4b32-aa66-bdff89af941e,Bros Chicken,B484H8HYR,2021-02-21,362.21,ERROR,DEV.SPEICUENTA NO PERTE A BCO,2021-02-23,2fdcf827-4e1f-4b32-aa66-bdff89af941e,2fdcf827-4e1f-4b32-aa66-bdff89af941e-362.21,ERROR_USUAL,B


In [10]:
query_error.shape

(157709, 13)

In [11]:
query_error.dtypes

type                         object
merchant_id                  object
merchant_name                object
report_id                    object
report_date_local    datetime64[ns]
amount                      float64
status                       object
error_description            object
ultimo_mov           datetime64[ns]
merchant_id_query            object
index                        object
esquema_query                object
ledger_flag                  object
dtype: object

### RUTA DE CUBETA ERROR

In [12]:
excel_error = pd.read_excel(r'C:\Users\Cristian.Aparicio\Documents\CUBETAS\2024\01. ENERO\CUBETA ERROR ENERO 2024.xlsx',
                            sheet_name='Cubeta ErrorLastMonth',converters={'report_id':str})
excel_error.columns.str.lower()
excel_error.rename(columns=lambda x: x.strip(),inplace=True)
excel_error['report_date_local']= pd.to_datetime(excel_error['report_date_local'])
excel_error['ultimo_mov']= pd.to_datetime(excel_error['ultimo_mov'])
excel_error['merchant_id_excel'] = excel_error['merchant_id']
excel_error['error_description'].fillna('sin_descripcion',inplace=True)
excel_error.loc[excel_error['error_description'].str.contains('Bank account information not available'), 'esquema_excel'] = 'SIN_CLABE'
excel_error.loc[~excel_error['error_description'].str.contains('Bank account information not available'), 'esquema_excel'] = 'ERROR_USUAL'
excel_error['ledger_flag'] =  excel_error['report_id'].astype(str).str[0]
excel_error.head()

,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_excel,esquema_query,esquema final,Unnamed: 14,merchant_id_excel,ledger_flag
0,32dd3cff-e387-4748-9a30-aae6d072391b,Herramientas Gaf,546699494,2020-07-27,781.92,ERROR,Cuenta cancelada,2020-07-27,7.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,32dd3cff-e387-4748-9a30-aae6d072391b,5
1,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,Momentos de Azucar,799855885,2020-08-05,251.54,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,7
2,2fb0b553-a383-4753-b48d-e0e80420d424,Abarrotes Salazar,983293644,2020-08-05,182.73,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,2fb0b553-a383-4753-b48d-e0e80420d424,9
3,aab0ed5c-4939-40b7-8226-cb3b85759e80,Hates dannys,222YTZXJG,2019-03-29,19.16,ERROR,FAILED 01042019,2019-04-01,4.00,"2,019.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,aab0ed5c-4939-40b7-8226-cb3b85759e80,2
4,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,Farmacia Los Angeles,2239M8G2N,2020-08-07,555.78,ERROR,Cuenta inexistente,2020-08-07,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,2


In [13]:
excel_error.shape

(163412, 17)

In [14]:
excel_error['esquema_excel'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [15]:
excel_error_settlement = excel_error
excel_error_settlement.head()

,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_excel,esquema_query,esquema final,Unnamed: 14,merchant_id_excel,ledger_flag
0,32dd3cff-e387-4748-9a30-aae6d072391b,Herramientas Gaf,546699494,2020-07-27,781.92,ERROR,Cuenta cancelada,2020-07-27,7.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,32dd3cff-e387-4748-9a30-aae6d072391b,5
1,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,Momentos de Azucar,799855885,2020-08-05,251.54,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,7
2,2fb0b553-a383-4753-b48d-e0e80420d424,Abarrotes Salazar,983293644,2020-08-05,182.73,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,2fb0b553-a383-4753-b48d-e0e80420d424,9
3,aab0ed5c-4939-40b7-8226-cb3b85759e80,Hates dannys,222YTZXJG,2019-03-29,19.16,ERROR,FAILED 01042019,2019-04-01,4.00,"2,019.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,aab0ed5c-4939-40b7-8226-cb3b85759e80,2
4,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,Farmacia Los Angeles,2239M8G2N,2020-08-07,555.78,ERROR,Cuenta inexistente,2020-08-07,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,2


In [16]:
excel_error_settlement.shape

(163412, 17)

In [17]:
excel_error_2 = excel_error_settlement[(excel_error_settlement['status_cubeta'] != 'SALIDA')]
excel_error_2['year'].unique()

array([2020., 2019., 2018., 2017., 2021., 2022., 2023., 2024.])

In [18]:
excel_error_2.dtypes

merchant_id                  object
merchant_name                object
report_id                    object
report_date_local    datetime64[ns]
amount                      float64
status                       object
error_description            object
ultimo_mov           datetime64[ns]
month                       float64
year                        float64
status_cubeta                object
esquema_excel                object
esquema_query        datetime64[ns]
esquema final                object
Unnamed: 14                  object
merchant_id_excel            object
ledger_flag                  object
dtype: object

In [19]:
excel_error_2.shape

(155949, 17)

## ANALISIS ERROR

### ANALISIS INFO MES ACTUAL

In [20]:
analisis_3 = query_error.merge(excel_error_2[['report_id','merchant_id_excel','esquema_excel']] ,on='report_id', how='left')
analisis_3['day'] = analisis_3['ultimo_mov'].dt.day
analisis_3['month'] = analisis_3['ultimo_mov'].dt.month
analisis_3['year'] = analisis_3['ultimo_mov'].dt.year
analisis_3['merchant_id_excel'].fillna('NEW',inplace=True)
analisis_3.loc[analisis_3['merchant_id_excel'] != 'NEW', "status_cubeta"] = 'ERROR'
analisis_3.loc[analisis_3['merchant_id_excel'] == 'NEW', "status_cubeta"] = 'NEW'
analisis_3.head()

,type,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,merchant_id_query,index,esquema_query,ledger_flag,merchant_id_excel,esquema_excel,day,month,year,status_cubeta
0,DAILY,858342a7-fd31-40a4-8cc1-8e360cb12698,FARMACIA FARMA FA,M289RR3NC,2022-11-06,1.80,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-11-07,858342a7-fd31-40a4-8cc1-8e360cb12698,858342a7-fd31-40a4-8cc1-8e360cb12698-1.8,ERROR_USUAL,M,858342a7-fd31-40a4-8cc1-8e360cb12698,ERROR_USUAL,7,11,2022,ERROR
1,DAILY,da3a4b3a-7b06-49f1-8f59-ae60feececdc,El rincon del cantante,TNDPS2WWB,2022-12-14,0.95,ERROR,Bank account information not available,2022-12-14,da3a4b3a-7b06-49f1-8f59-ae60feececdc,da3a4b3a-7b06-49f1-8f59-ae60feececdc-0.95,SIN_CLABE,T,da3a4b3a-7b06-49f1-8f59-ae60feececdc,SIN_CLABE,14,12,2022,ERROR
2,INSTANT,43685a7e-2583-480a-94be-f6aa7cfee8f6,NAILKERY SAN ANGELO,FYMY32LCT,2021-12-26,373.71,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2021-12-28,43685a7e-2583-480a-94be-f6aa7cfee8f6,43685a7e-2583-480a-94be-f6aa7cfee8f6-373.71,ERROR_USUAL,F,43685a7e-2583-480a-94be-f6aa7cfee8f6,ERROR_USUAL,28,12,2021,ERROR
3,DAILY,3681aa22-8083-43a4-a552-ad7372081221,Bloombelle Lashes,RPB8JVFZD,2018-09-03,47.91,ERROR,failed swap,2018-09-03,3681aa22-8083-43a4-a552-ad7372081221,3681aa22-8083-43a4-a552-ad7372081221-47.91,ERROR_USUAL,R,3681aa22-8083-43a4-a552-ad7372081221,ERROR_USUAL,3,9,2018,ERROR
4,DAILY,2fdcf827-4e1f-4b32-aa66-bdff89af941e,Bros Chicken,B484H8HYR,2021-02-21,362.21,ERROR,DEV.SPEICUENTA NO PERTE A BCO,2021-02-23,2fdcf827-4e1f-4b32-aa66-bdff89af941e,2fdcf827-4e1f-4b32-aa66-bdff89af941e-362.21,ERROR_USUAL,B,2fdcf827-4e1f-4b32-aa66-bdff89af941e,ERROR_USUAL,23,2,2021,ERROR


In [21]:
analisis_3.shape

(157709, 19)

In [22]:
analisis_3.to_csv('review.csv',index=False)

### DATA FINAL

In [23]:
analisis_3.columns

Index(['type', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'merchant_id_query', 'index', 'esquema_query',
       'ledger_flag', 'merchant_id_excel', 'esquema_excel', 'day', 'month',
       'year', 'status_cubeta'],
      dtype='object')

In [24]:
error_info_actual = analisis_3[['merchant_id','merchant_name','report_id','report_date_local','amount','status'
                                ,'error_description', 'ultimo_mov','month','year','status_cubeta','esquema_query']]
error_info_actual.head()

,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query
0,858342a7-fd31-40a4-8cc1-8e360cb12698,FARMACIA FARMA FA,M289RR3NC,2022-11-06,1.80,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-11-07,11,2022,ERROR,ERROR_USUAL
1,da3a4b3a-7b06-49f1-8f59-ae60feececdc,El rincon del cantante,TNDPS2WWB,2022-12-14,0.95,ERROR,Bank account information not available,2022-12-14,12,2022,ERROR,SIN_CLABE
2,43685a7e-2583-480a-94be-f6aa7cfee8f6,NAILKERY SAN ANGELO,FYMY32LCT,2021-12-26,373.71,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2021-12-28,12,2021,ERROR,ERROR_USUAL
3,3681aa22-8083-43a4-a552-ad7372081221,Bloombelle Lashes,RPB8JVFZD,2018-09-03,47.91,ERROR,failed swap,2018-09-03,9,2018,ERROR,ERROR_USUAL
4,2fdcf827-4e1f-4b32-aa66-bdff89af941e,Bros Chicken,B484H8HYR,2021-02-21,362.21,ERROR,DEV.SPEICUENTA NO PERTE A BCO,2021-02-23,2,2021,ERROR,ERROR_USUAL


In [25]:
error_info_actual.shape

(157709, 12)

In [26]:
error_info_actual.to_csv('info_actual.csv',index=False)

### PIVOT

In [27]:
pivot_error_actual = pd.pivot_table(error_info_actual,index=['year'],columns='status_cubeta'
                                        ,values=['amount'],aggfunc=np.sum,margins=True).reset_index()
pivot_error_actual.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\1706156035.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_error_actual = pd.pivot_table(error_info_actual,index=['year'],columns='status_cubeta'
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\1706156035.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_error_actual = pd.pivot_table(error_info_actual,index=['year'],columns='status_cubeta'
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\1706156035.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is cu

year        amount                  
status_cubeta               ERROR NEW           All
0              2017  1,429,808.45 NaN  1,429,808.45
1              2018  2,980,860.37 NaN  2,980,860.37
2              2019  5,416,983.11 NaN  5,416,983.11
3              2020  8,207,094.13 NaN  8,207,094.13
4              2021 27,782,992.55 NaN 27,782,992.55

In [28]:
pivot_error_actual.columns = pivot_error_actual.columns.droplevel(0)

In [29]:
pivot_error_actual.columns =['year','error','new','all'] 
pivot_error_actual.tail()

,year,error,new,all
4,2021,"27,782,992.55",NaN,"27,782,992.55"
5,2022,"28,407,079.83",NaN,"28,407,079.83"
6,2023,"48,043,446.91",NaN,"48,043,446.91"
7,2024,"936,674.22","12,641,816.80","13,578,491.02"
8,All,"123,204,939.57","12,641,816.80","135,846,756.37"


In [30]:
pivot_error_actual.shape

(9, 4)

### NEW ONLY

In [31]:
new_error = error_info_actual[(error_info_actual['status_cubeta'] == 'NEW')]
new_error.head()

,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query
12,c9a433cf-634d-4f17-bcd3-5cd0a1eb5e84,NaN,A5SF5ZV54,2024-01-06,4.79,ERROR,Bank account information not available,2024-01-06,1,2024,NEW,SIN_CLABE
17,452bd855-7343-4c73-9711-0b62a0acfdb9,NaN,AMHL9RNKG,2024-01-30,114.99,ERROR,X2,2024-01-30,1,2024,NEW,ERROR_USUAL
46,ce47451d-1b7f-424e-8606-c6d34df5f790,NaN,ACB4JVPM8,2024-01-17,"3,219.68",ERROR,/21/CB Return Exceeds the limit of credits all...,2024-01-17,1,2024,NEW,ERROR_USUAL
69,d95205c2-9b18-4a3a-b098-88b36e9f37fa,NaN,A2C8XZDMK,2024-01-30,792.46,ERROR,/21/CB Return Exceeds authorized account balan...,2024-01-30,1,2024,NEW,ERROR_USUAL
70,1d10d0b3-0852-40f5-9ea8-962e98d310d7,NaN,ARC9QV22Q,2024-01-30,389.99,ERROR,DEVOLUCION(20): Excede el limite de saldo auto...,2024-01-30,1,2024,NEW,ERROR_USUAL


In [32]:
new_error.shape

(9783, 12)

### ANALISIS HISTORICO

In [33]:
excel_error_2.shape

(155949, 17)

In [34]:
excel_error_2['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

In [35]:
analisis_4 = excel_error_2.merge(query_error[['report_id','merchant_id_query','ultimo_mov']] ,on='report_id', how='left')
#analisis_4['day'] = analisis_4['ultimo_mov'].dt.day

analisis_4['merchant_id_query'].fillna('SALIDA',inplace=True)
analisis_4.loc[analisis_4['ultimo_mov_y'].notnull(), "ultimo_mov_final"] = analisis_4['ultimo_mov_y']
analisis_4.loc[analisis_4['ultimo_mov_y'].isnull(), "ultimo_mov_final"] = analisis_4['ultimo_mov_x']
analisis_4.loc[analisis_4['merchant_id_query'] != 'SALIDA', "status_cubeta"] = 'ERROR'
analisis_4.loc[analisis_4['merchant_id_query'] == 'SALIDA', "status_cubeta"] = 'SALIDA'
analisis_4['month'] = analisis_4['ultimo_mov_final'].dt.month
analisis_4['year'] = analisis_4['ultimo_mov_final'].dt.year
analisis_4.head()

,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov_x,month,year,status_cubeta,esquema_excel,esquema_query,esquema final,Unnamed: 14,merchant_id_excel,ledger_flag,merchant_id_query,ultimo_mov_y,ultimo_mov_final
0,32dd3cff-e387-4748-9a30-aae6d072391b,Herramientas Gaf,546699494,2020-07-27,781.92,ERROR,Cuenta cancelada,2020-07-27,7.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,32dd3cff-e387-4748-9a30-aae6d072391b,5,32dd3cff-e387-4748-9a30-aae6d072391b,2020-07-27,2020-07-27
1,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,Momentos de Azucar,799855885,2020-08-05,251.54,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,7,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,2020-08-05,2020-08-05
2,2fb0b553-a383-4753-b48d-e0e80420d424,Abarrotes Salazar,983293644,2020-08-05,182.73,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,2fb0b553-a383-4753-b48d-e0e80420d424,9,2fb0b553-a383-4753-b48d-e0e80420d424,2020-08-05,2020-08-05
3,aab0ed5c-4939-40b7-8226-cb3b85759e80,Hates dannys,222YTZXJG,2019-03-29,19.16,ERROR,FAILED 01042019,2019-04-01,4.00,"2,019.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,aab0ed5c-4939-40b7-8226-cb3b85759e80,2,aab0ed5c-4939-40b7-8226-cb3b85759e80,2019-04-01,2019-04-01
4,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,Farmacia Los Angeles,2239M8G2N,2020-08-07,555.78,ERROR,Cuenta inexistente,2020-08-07,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,2,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,2020-08-07,2020-08-07


In [36]:
analisis_4.shape

(155949, 20)

In [37]:
analisis_4.to_csv('info_historico.csv',index=False)

In [38]:
analisis_4['year'].unique()

array([2020., 2019., 2018., 2017., 2021., 2022., 2023., 2024.,   nan])

### DATA FINAL

In [39]:
error_limpio = analisis_4[['merchant_id','merchant_name','report_id','report_date_local','amount','status'
                                ,'error_description', 'ultimo_mov_final','month','year','status_cubeta','esquema_excel']]
error_limpio['amount'].fillna(0,inplace=True)
error_limpio.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\2952077184.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_limpio['amount'].fillna(0,inplace=True)


,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov_final,month,year,status_cubeta,esquema_excel
0,32dd3cff-e387-4748-9a30-aae6d072391b,Herramientas Gaf,546699494,2020-07-27,781.92,ERROR,Cuenta cancelada,2020-07-27,7.00,"2,020.00",ERROR,ERROR_USUAL
1,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,Momentos de Azucar,799855885,2020-08-05,251.54,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL
2,2fb0b553-a383-4753-b48d-e0e80420d424,Abarrotes Salazar,983293644,2020-08-05,182.73,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL
3,aab0ed5c-4939-40b7-8226-cb3b85759e80,Hates dannys,222YTZXJG,2019-03-29,19.16,ERROR,FAILED 01042019,2019-04-01,4.00,"2,019.00",ERROR,ERROR_USUAL
4,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,Farmacia Los Angeles,2239M8G2N,2020-08-07,555.78,ERROR,Cuenta inexistente,2020-08-07,8.00,"2,020.00",ERROR,ERROR_USUAL


In [40]:
error_limpio.shape

(155949, 12)

In [41]:
error_historico = pd.concat([error_limpio,new_error]).reset_index()
error_historico.loc[(error_historico['esquema_excel'].notnull()), 'esquema_final'] = error_historico['esquema_excel']
error_historico.loc[(error_historico['esquema_query'].notnull()), 'esquema_final'] = error_historico['esquema_query']
error_historico.head()

,index,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov_final,month,year,status_cubeta,esquema_excel,ultimo_mov,esquema_query,esquema_final
0,0,32dd3cff-e387-4748-9a30-aae6d072391b,Herramientas Gaf,546699494,2020-07-27,781.92,ERROR,Cuenta cancelada,2020-07-27,7.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
1,1,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,Momentos de Azucar,799855885,2020-08-05,251.54,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
2,2,2fb0b553-a383-4753-b48d-e0e80420d424,Abarrotes Salazar,983293644,2020-08-05,182.73,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
3,3,aab0ed5c-4939-40b7-8226-cb3b85759e80,Hates dannys,222YTZXJG,2019-03-29,19.16,ERROR,FAILED 01042019,2019-04-01,4.00,"2,019.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
4,4,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,Farmacia Los Angeles,2239M8G2N,2020-08-07,555.78,ERROR,Cuenta inexistente,2020-08-07,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL


In [42]:
error_historico.shape

(165732, 16)

### PIVOT POR AÑO

In [43]:
pivot_error_historico = pd.pivot_table(error_historico,index=['year'],
                                       columns='status_cubeta',
                                       values=['amount'],
                                       aggfunc=np.sum,
                                       margins=True).reset_index()
pivot_error_historico.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\964866664.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_error_historico = pd.pivot_table(error_historico,index=['year'],
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\964866664.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_error_historico = pd.pivot_table(error_historico,index=['year'],
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\964866664.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future

year        amount                            
status_cubeta                  ERROR NEW    SALIDA           All
0             2,017.00  1,429,808.45 NaN       NaN  1,429,808.45
1             2,018.00  2,980,860.37 NaN 11,221.53  2,992,081.90
2             2,019.00  5,416,983.11 NaN  1,407.66  5,418,390.77
3             2,020.00  8,207,094.13 NaN  8,617.10  8,215,711.23
4             2,021.00 27,782,992.55 NaN 66,918.42 27,849,910.97

In [44]:
pivot_error_historico.columns = pivot_error_historico.columns.droplevel(0)

In [45]:
pivot_error_historico.columns =['year','error','new','salida','all'] 
pivot_error_historico.fillna(0,inplace=True)
pivot_error_historico['sum_tabla_final'] = pivot_error_historico['error'] + pivot_error_historico['salida']
pivot_error_historico.head()

,year,error,new,salida,all,sum_tabla_final
0,"2,017.00","1,429,808.45",0.00,0.00,"1,429,808.45","1,429,808.45"
1,"2,018.00","2,980,860.37",0.00,"11,221.53","2,992,081.90","2,992,081.90"
2,"2,019.00","5,416,983.11",0.00,"1,407.66","5,418,390.77","5,418,390.77"
3,"2,020.00","8,207,094.13",0.00,"8,617.10","8,215,711.23","8,215,711.23"
4,"2,021.00","27,782,992.55",0.00,"66,918.42","27,849,910.97","27,849,910.97"


In [46]:
pivot_error_historico.shape

(9, 6)

### PIVOT POR ESQUEMA

In [47]:
pivot_error_historico_esquema = pd.pivot_table(error_historico,index=['esquema_final'],
                                               columns='status_cubeta',
                                               values=['amount'],
                                               aggfunc=np.sum,
                                               margins=True).reset_index()
pivot_error_historico_esquema.columns = pivot_error_historico_esquema.columns.droplevel(0)
pivot_error_historico_esquema.columns =['esquema_final','error','new','salida','all'] 
pivot_error_historico_esquema.fillna(0,inplace=True)
pivot_error_historico_esquema['sum_tabla_final'] = pivot_error_historico_esquema['error'] + pivot_error_historico_esquema['salida']
pivot_error_historico_esquema

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\2618817985.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_error_historico_esquema = pd.pivot_table(error_historico,index=['esquema_final'],
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\2618817985.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_error_historico_esquema = pd.pivot_table(error_historico,index=['esquema_final'],
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\2618817985.py:1: FutureWarning: The provided callable <function sum at 0x00000240E2EFEE50> is currently us

,esquema_final,error,new,salida,all,sum_tabla_final
0,ERROR_USUAL,"94,697,873.05","9,854,739.47","11,226,114.78","115,778,727.30","105,923,987.83"
1,SIN_CLABE,"28,507,066.52","2,787,077.33","4,012,491.46","35,306,635.31","32,519,557.98"
2,All,"123,204,939.57","12,641,816.80","15,238,606.24","151,085,362.61","138,443,545.81"


In [48]:
pivot_error_historico_esquema.shape

(3, 6)

### INTEGRACION

In [49]:
integracion_1 = error_historico[(error_historico['status_cubeta'] == 'SALIDA')]
integracion_1['amount'] = integracion_1['amount']*-1
integracion_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_21328\1772267744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  integracion_1['amount'] = integracion_1['amount']*-1


,index,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov_final,month,year,status_cubeta,esquema_excel,ultimo_mov,esquema_query,esquema_final
1340,1340,40e6cbb5-b68e-4018-aef9-f2ec23cac876,Carim,5L3SKD9ZZ,2019-10-28,-114.99,ERROR,sin_descripcion,2019-10-28,10.00,"2,019.00",SALIDA,ERROR_USUAL,NaT,NaN,ERROR_USUAL
2230,2230,ad78e3bc-442d-4498-b49a-091697ffa5bc,YAAB NUTRITION CENTER,83N7ZWD8W,2020-05-24,"-1,147.01",ERROR,BANCO RECEPTOR ESTA DESCONECTADO,2020-05-24,5.00,"2,020.00",SALIDA,ERROR_USUAL,NaT,NaN,ERROR_USUAL
2241,2241,c4d77042-ce1b-4094-87cc-61c0bc58c356,PERFUMES EUROPEOS TX,842DJWS6B,2020-02-13,-178.23,ERROR,sin_descripcion,2020-02-13,2.00,"2,020.00",SALIDA,ERROR_USUAL,NaT,NaN,ERROR_USUAL
4318,4318,40e6cbb5-b68e-4018-aef9-f2ec23cac876,Carim,FR9VSK776,2019-10-14,-238.61,ERROR,sin_descripcion,2019-10-14,10.00,"2,019.00",SALIDA,ERROR_USUAL,NaT,NaN,ERROR_USUAL
4356,4356,8918067f-191b-4997-82a2-93b9929526b5,MINISUPER HOLANDA,FTGZ4W9GP,2020-06-01,-99.70,ERROR,sin_descripcion,2020-06-01,6.00,"2,020.00",SALIDA,ERROR_USUAL,NaT,NaN,ERROR_USUAL


In [50]:
integracion_1.shape

(8023, 16)

In [51]:
integracion_final = pd.concat([error_historico,integracion_1])
integracion_final.head()

,index,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov_final,month,year,status_cubeta,esquema_excel,ultimo_mov,esquema_query,esquema_final
0,0,32dd3cff-e387-4748-9a30-aae6d072391b,Herramientas Gaf,546699494,2020-07-27,781.92,ERROR,Cuenta cancelada,2020-07-27,7.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
1,1,f391cdb2-9901-4a0b-bd3e-420e6c641c4d,Momentos de Azucar,799855885,2020-08-05,251.54,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
2,2,2fb0b553-a383-4753-b48d-e0e80420d424,Abarrotes Salazar,983293644,2020-08-05,182.73,ERROR,Excede el limite de saldo auto,2020-08-05,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
3,3,aab0ed5c-4939-40b7-8226-cb3b85759e80,Hates dannys,222YTZXJG,2019-03-29,19.16,ERROR,FAILED 01042019,2019-04-01,4.00,"2,019.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL
4,4,dfda38a1-2cfb-4e12-9f4d-b17bc8129ffc,Farmacia Los Angeles,2239M8G2N,2020-08-07,555.78,ERROR,Cuenta inexistente,2020-08-07,8.00,"2,020.00",ERROR,ERROR_USUAL,NaT,NaN,ERROR_USUAL


In [52]:
integracion_final.shape

(173755, 16)

# EXCEL FINAL 

# ERROR

In [53]:
writer = pd.ExcelWriter('DATA ERROR.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_error_historico.to_excel(writer, sheet_name='PIVOT POR AÑO', index=False)
pivot_error_historico_esquema.to_excel(writer, sheet_name='PIVOT POR ESQUEMA', index=False)
error_historico.to_excel(writer, sheet_name='ERROR', index=False)
error_info_actual.to_excel(writer, sheet_name='INFO MES ACTUAL', index=False)
integracion_final.to_excel(writer, sheet_name='INTEGRACION', index=False)
writer.close()

# DATA ERROR PARA POLIZA CONTABLE FASE III.

In [54]:
error_info_actual.to_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2024\01. ENERO\DATA\ERROR ENERO 2024.csv')